In [22]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from calendar import monthrange

In [23]:
def get_link(month : int, year : int, city='Kursk') -> str:
    if city == 'Kursk':
        city_id = 34009 #Адресс Места Роста Сои в Курск
    elif city == 'Voronez':
        city_id = 34238 #Адресс Места Роста Сои в Воронеже 
    day_start, day_end = monthrange(year, month)
    
    return "http://pogodaiklimat.ru/weather.php?id=" + f"{city_id}" + "&bday=" + f"{day_start}" + "&fday=" + f"{day_end}" + "&amonth=" + f"{month}" + "&ayear=" + f"{year}" + "&bot=2"

In [24]:
def get_pred_single(month, year, city='Kursk'):
    if city == 'Kursk':
        page = requests.get(get_link(month, year, city='Kursk'))
    elif city == 'Voronez':
        page = requests.get(get_link(month, year, city='Voronez'))        

    page.encoding = 'utf-8'
    html = page.text

    soup = bs(html, 'html.parser')

    data_1 = []

    rows = soup.table.find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        data_1.append([column.text for column in columns])

    data_table = pd.DataFrame(np.array(data_1[1:]).astype(float), columns=data_1[0][0].replace(',', '').split('\t'))

    data_2 = []

    rows = soup.find_all('tr')

    for row in rows[len(data_1):]:
        columns = row.find_all('td')
        data_2.append([column.text for column in columns])

    headers = [0] * 18 
    headers[:1] = data_2[0][0].split()
    headers[2:] = data_2[0][1:]

    weather_table =  pd.DataFrame(np.array(data_2[1:]), columns=headers)

    return pd.merge(data_table, weather_table, left_index=True, right_index=True)

In [25]:
kursk_9_23 = get_pred_single(9, 2023)

In [26]:
kursk_9_23

,Время (UTC),дата,Ветер,"(напр.,м/с)",Видим.,Явления,Облачность,Т(С),Тd(С),f(%),Тe(С),Тes(С),Комфортность,P(гПа),Po(гПа),Тmin(С),Tmax(С),R(мм),R24(мм),S(см)
0,0.0,4.09,С,2,10 км,,ясно,+15.0,+9.4,69,+15,+15,тепло,1020.0,990.7,,,,,
1,3.0,4.09,С,1,10 км,,5/0[Ci fib],+13.7,+9.4,75,+14,+14,по сезону,1020.1,990.7,+13.7,,,,
2,6.0,4.09,С,1,10 км,,8/0[Ci fib],+17.4,+11.2,67,+18,+23,тепло,1020.1,991.0,+13.6,,,,
3,9.0,4.09,С,3,10 км,,9/3 1000 м[Cu hum Ac trans Ci fib],+22.2,+10.7,48,+24,+28,комфорт,1019.8,991.2,,,,,
4,12.0,4.09,С,3,10 км,,"9/5 1000 м[Sc diur, vesp Ac trans Ci fib]",+22.8,+10.3,45,+25,+26,комфорт,1019.7,991.2,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,9.0,30.09,З,2,10 км,,ясно,+21.6,+7.9,41,+23,+29,комфорт,1021.3,992.6,,,,,
212,12.0,30.09,ЮЗ,2,10 км,,ясно,+22.8,+8.0,39,+24,+28,комфорт,1019.9,991.4,,,,,
213,15.0,30.09,штиль,0,10 км,,ясно,+19.1,+9.7,54,+20,+20,тепло,1018.8,990.0,,,,,
214,18.0,30.09,штиль,0,10 км,,ясно,+15.0,+9.3,69,+15,+15,тепло,1019.0,989.7,,+23.3,,,


In [27]:
kursk_1_22 =  get_pred_single(1, 2022)

In [28]:
kursk_1_22

,Время (UTC),дата,Ветер,"(напр.,м/с)",Видим.,Явления,Облачность,Т(С),Тd(С),f(%),Тe(С),Тes(С),Комфортность,P(гПа),Po(гПа),Тmin(С),Tmax(С),R(мм),R24(мм),S(см)
0,0.0,5.01,ЮВ,2,1000 м,дымка {ливн. снег},10/10 200 м[St],-3.8,-4.4,96,-8,-8,по сезону,1005.8,974.9,,,,,
1,3.0,5.01,ЮВ,3,500 м,переохл. туман,?/?,-2.3,-2.8,96,-8,-8,по сезону,1003.4,972.7,-6.8,,0.3,,
2,6.0,5.01,ЮВ,5,1000 м,слаб. ливневой снег {туман} [диам. сложн. отло...,10/10 600 м[Cb cap],-0.3,-0.8,96,-7,-7,по сезону,999.3,969.0,-6.3,,1,,19
3,9.0,5.01,Ю,5-10 {12},2000 м,слаб. ливневой дождь {метель},10/10 600 м[Cb cap],+1.5,+1.0,96,-4,-4,тепло,995.9,965.9,,,,,
4,12.0,5.01,ЮЗ,5-10 {12},2000 м,слаб. дождь {ливн. дождь},10/10 200 м[Frnb Ns],+2.4,+1.9,96,-3,-3,тепло,995.7,965.8,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,9.0,31.01,Ю,5 {11},2000 м,снег {ливн. снег},10/10 300 м[Frnb Ns],-0.7,-1.7,93,-7,-7,по сезону,990.9,960.7,,,,,
212,12.0,31.01,Ю,3 {10},2000 м,слаб. ливневой снег,10/10 600 м[Cb cap],-0.1,-1.0,94,-5,-5,по сезону,990.3,960.2,,,,,
213,15.0,31.01,ЮЗ,4,4000 м,слаб. снег {ливн. снег} [диам. отлож. мокр. сн...,10/10 300 м[Frnb Ns],-0.8,-1.2,97,-7,-7,по сезону,991.8,961.6,,,4,,
214,18.0,31.01,ЮЗ,3,10 км,"{ливн. снег, снег}",10/10 600 м[Sc],-0.9,-1.4,96,-6,-6,по сезону,994.2,963.9,,-0.1,3,,
